In [26]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score

df = pd.read_csv('/content/dataset3 (1).psv')

nova_coluna = df[' employee_id|department|region|education|gender|recruitment_channel|no_of_trainings|age|previous_year_rating|length_of_service|awards_won?|avg_training_score|is_promoted'].str.split('|', expand=True)
df[nova_coluna.columns] = nova_coluna

df = pd.DataFrame(nova_coluna)
df.columns = ['employee_id', 'department', 'region', 'education', 'gender', 'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating', 'length_of_service', 'awards_won?', 'avg_training_score', 'is_promoted']


df['gender'].replace(['m', 'eu sou homem hetero homossexual!'], 'MALE', inplace=True)
df['gender'].replace(['f', 'fEMALE'], 'FEMALE', inplace=True)


df['education'] = df['education'].str.strip().replace('', 'UNKNOWN')
df['department'].replace(['Legal', 'quem trouxe quenga'], 'UNKOWN', inplace = True)
df['region'].replace(['casa da lu na suecia'], 'UNKOWN', inplace = True)


df['age'] = df['age'].astype(int)
df['length_of_service'] = df['length_of_service'].astype(int)


df['no_of_trainings'] = df['no_of_trainings'].str.strip().replace('', '0.0')
df['no_of_trainings'] = df['no_of_trainings'].astype(float)



df['previous_year_rating'] = df['previous_year_rating'].str.strip().replace('', '0.0')
df['previous_year_rating'] = df['previous_year_rating'].astype(float)

df['avg_training_score'] = df['avg_training_score'].str.strip().replace('', '0.0')
df['avg_training_score'] = df['avg_training_score'].astype(float)

df.set_index("employee_id", inplace=True)

import numpy as np


linhas_incorretas = df[df['age'] == -32].index



df.loc[linhas_incorretas, 'age'] = 32
valor_incorreto = 123456
df['age'] = df['age'].replace(valor_incorreto, np.nan)

from sklearn.preprocessing import OrdinalEncoder


coluna_department = df[['department']]


ordinal_encoder = OrdinalEncoder()


coluna_codificada = ordinal_encoder.fit_transform(coluna_department)
coluna_codificada = coluna_codificada.astype(int)

df['department_encoded'] = coluna_codificada




coluna_region = df[['region']]


ordinal_encoder_r = OrdinalEncoder()


coluna_region = ordinal_encoder_r.fit_transform(coluna_region)
coluna_region = coluna_region.astype(int)

df['region_encoded'] = coluna_region

coluna_education  = df[['education']]

ordinal_encoder_e = OrdinalEncoder()


coluna_education = ordinal_encoder_e.fit_transform(coluna_education)
coluna_education = coluna_education.astype(int)

df['education_encoded'] = coluna_education



coluna_gender  = df[['gender']]


ordinal_encoder_g = OrdinalEncoder()


coluna_gender = ordinal_encoder_g.fit_transform(coluna_gender)
coluna_education = coluna_education.astype(int)
coluna_gender = coluna_gender.astype(int)

df['gender_encoded'] = coluna_gender


df['age'].fillna(df['age'].mean(), inplace=True)

df['no_of_trainings'].fillna(0, inplace=True)



coluna_recruitment = df[['recruitment_channel']]


ordinal_encoder_re = OrdinalEncoder()


coluna_recruitment = ordinal_encoder_re.fit_transform(coluna_recruitment)

coluna_recruitment = coluna_recruitment.astype(int)


df['recruitment_channel_encoded'] = coluna_recruitment


colunas_antigas = ['department', 'region', 'education', 'gender', 'recruitment_channel']  # Substitua pelos nomes reais das colunas que você deseja dropar
df.drop(colunas_antigas, axis=1, inplace=True)


df.rename(columns={'department_encoded': 'department'}, inplace=True)
df.rename(columns={'region_encoded': 'region'}, inplace=True)
df.rename(columns={'education_encoded': 'education'}, inplace=True)
df.rename(columns={'gender_encoded': 'gender'}, inplace=True)
df.rename(columns={'recruitment_channel_encoded': 'recruitment_channel'}, inplace=True)


nova_ordem = ['department', 'region', 'education', 'gender', 'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating', 'length_of_service', 'awards_won?', 'avg_training_score', 'is_promoted']  # Substitua pelos nomes reais das colunas e a nova ordem desejada


df = df[nova_ordem]


from sklearn.preprocessing import MinMaxScaler

colunas_continuas = ['no_of_trainings', 'age', 'previous_year_rating', 'length_of_service', 'awards_won?', 'avg_training_score']


scaler = MinMaxScaler()


df[colunas_continuas] = scaler.fit_transform(df[colunas_continuas])
df_nova = pd.DataFrame(df)

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier


X = df.drop('is_promoted', axis=1)
y = df['is_promoted']



X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2, stratify=y)


from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

valores_k = range(1, 21)
k_melhor_valor = None
melhor_acuracia = 0.0

for k in valores_k:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    acuracia = accuracy_score(y_test, y_pred)

    if acuracia > melhor_acuracia:
        melhor_acuracia = acuracia
        k_melhor_valor = k

print("Melhor valor para K:", k_melhor_valor)
print("Acurácia dos dados de teste:", melhor_acuracia)

Melhor valor para K: 4
Acurácia dos dados de teste: 0.9161649334063127


In [27]:

knn = KNeighborsClassifier(n_neighbors=k_melhor_valor)
knn.fit(X_train, y_train)
y_pred_probs = knn.predict_proba(X_test)[:, 1]
y_pred = (y_pred_probs > 0.5).astype(int)


y_test_int = y_test.astype(int)


Precisão = precision_score(y_test_int, y_pred)
Recall = recall_score(y_test_int, y_pred)


print("Precisão:", Precisão)
print("Recall:", Recall)



Precisão: 0.5925925925925926
Recall: 0.05139186295503212


B) Tem como fazer algumas suposições iniciais, elas são: 'previous_year_rating', enquanto a classificação do funcionário no ano anterior, pode ser um indicador importante para a promoção.'awards_won?', enquanto um indicador dos funcionários que ganharam prêmios, pode estar relacionado a ser mais propensos a promoção. 'avg_training_score', enquanto funcionários com pontuações mais altas nos treinamentos, pode viabilizar uma maior probabilidade de promoção. Já colunas como o 'gender' parecem não ter muita relação com a possivel promoção do funcionário.

F) Lendo um pouco a respeito de dados desbalanceados, descobri que referem-se a um cenário em que as classes ou categorias de interesse em um conjunto de dados não estão representadas de forma equilibrada. Em outras palavras, uma ou mais classes têm um número muito maior de exemplos do que outras. Suponho que no caso do meu modelo, tenha relação com o Viés de classes: "Os modelos treinados em conjuntos de dados desbalanceados tendem a apresentar um viés em direção às classes majoritárias. Isso ocorre porque o modelo aprende a prever a classe majoritária com mais precisão devido à maior quantidade de exemplos disponíveis, enquanto as classes minoritárias são sub-representadas e podem ser erroneamente classificadas." Além disso, o KNN pode não ser ideial por assumir distâncias euclidianas entre variáveis para calcular os vizinhos mais próximos.
Na teoria, a acurácia é ótima, mas quando vamos analisar a precisão e Recall, notamos que há um problema.


G) Precisão: 0.5925925925925926 (59.26%)
Recall: 0.05139186295503212 (5.14%)
Devemos considerar que: o valor da precição indica que aproximadamente 59.26% das "instâncias classificadas como positivas" são verdadeiros positivos, enquanto o restante, 40.74% , são falsos positivos. O modelo está classificando corretamente a maioria das instâncias positivas, mas está cometendo muitos falsos positivos. Já quanto ao Recall: Um valor de 5.14% indica que o modelo é capaz de identificar corretamente apenas aproximadamente 5.14% das instâncias positivas no conjunto de dados. Isso significa que o modelo está perdendo a maioria das instâncias positivas e apresenta um alto número de falsos negativos.
 O valor de recall tão baixos é problemático, pois indica que o modelo está falhando em identificar a maioria das instâncias positivas, resultando em uma baixa capacidade de detecção.
Por outro lado, a precisão é mais alta, o que indica que quando o modelo classifica uma instância como positiva, ele tem uma probabilidade relativamente maior de estar correto.
Assim, os resultados atingidos não são nem de longe ideias.

In [28]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from imblearn.over_sampling import SMOTE



warnings.filterwarnings("ignore")


smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


brf_modelo = BalancedRandomForestClassifier(random_state=42)
brf_modelo.fit(X_train_resampled, y_train_resampled)


warnings.filterwarnings("default")


y_pred_prob = brf_modelo.predict_proba(X_test)[:, 1]


threshold = 0.4
y_pred_adjusted = (y_pred_prob > threshold).astype(int)

y_test_int = y_test.astype(int)


acuracia = accuracy_score(y_test_int, y_pred_adjusted)
Precisão = precision_score(y_test_int, y_pred_adjusted)
Recall = recall_score(y_test_int, y_pred_adjusted)

print("Acurácia nos dados de teste:", acuracia)
print("Precisão:", Precisão)
print("Recall:", Recall)


Acurácia nos dados de teste: 0.9051268016785258
Precisão: 0.43552311435523117
Recall: 0.38329764453961457
